![](https://images.pexels.com/photos/2092718/pexels-photo-2092718.jpeg?auto=compress&cs=tinysrgb&h=750&w=1260)

# Classes with `attr.slots` and `attr.frozen`

Kevin J. Walchko

25 Apr 2020

---

After writing a lot of python and classes, `attr` module is *attractive* because
it promises the ability to write less and provide some simple things that make
`collections.namedtuple` *attractive*.

- `slots`: increase memory efficiency of the class
- `frozen`: produce an immutable data class like a `namedtuple`

This is just me playing around with 2 attractive aspects of `attr`.

I am going to use `timeit` and save off a few simple runs. The data structure gives me:

```python
dir(timeit) =>
[...
 'all_runs',
 'average',
 'best',
 'compile_time',
 'loops',
 'repeat',
 'stdev',
 'timings',
 'worst']
```

In [61]:
def go(p=False):
    m = (1,2,3,4,5,6)
    a = Vec(1,2,3)
    for i in a:
        x = i + 5
    assert a + (1,2,3) == Vec(2,4,6)
    assert a + m[3:] == Vec(5,7,9)
    assert a.km == (0.001,0.002,0.003)

In [104]:
# resets everything
results = {}
states = [(False, False), (True, False), (False, True), (True, True)]
cnt = 0

In [90]:
import attr
from pprint import pprint
import pandas as pd

In [119]:
s, f = states[cnt]
cnt = (cnt + 1)%4

@attr.s(slots=s, frozen=f)
class Vec:
    x=attr.ib()
    y=attr.ib()
    z=attr.ib()
    def __add__(self, b):
        if isinstance(b, tuple) and len(b) >= 3:
            x = self.x + b[0]
            y = self.y + b[1]
            z = self.z + b[2]
        else:
            x = self.x + b.x
            y = self.y + b.y
            z = self.z + b.z
        return Vec(x,y,z)
    def __iter__(self):
        for i in (self.x, self.y, self.z):
            yield i
    @property
    def km(self):
        return (self.x/1000, self.y/1000, self.z/1000)
    
a = %timeit -o go()
print(f"slots: {s}   frozen: {f}")
results[(s,f)] = (a, Vec(1000,-2.36,33e-19).__sizeof__())
go(True)

7.12 µs ± 184 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
slots: True   frozen: True


In [120]:
pprint(results)

{(False, False): (<IPython.core.magics.execution.TimeitResult object at 0x7f9ac1dbb690>,
                  32),
 (False, True): (<IPython.core.magics.execution.TimeitResult object at 0x7f9aedf84210>,
                 32),
 (True, False): (<IPython.core.magics.execution.TimeitResult object at 0x7f9ac1e2d290>,
                 48),
 (True, True): (<IPython.core.magics.execution.TimeitResult object at 0x7f9ac1dbbcd0>,
                48)}


In [121]:
# build a list of performance
# ave = usec
# stdev = nsec
m = [[s,f,a.average*1e6, a.stdev*1e6, sz] for (s,f),(a,sz) in results.items()]

In [122]:
m

[[False, False, 5.553422401260053, 0.3225070696245372, 32],
 [False, True, 5.630893077302193, 0.20103028908316836, 32],
 [True, True, 7.118934657025551, 0.1838108938731178, 48],
 [True, False, 5.321727893980486, 0.23522055508820028, 48]]

In [123]:
df = pd.DataFrame(m, columns=['slots','frozen','ave','stdev','size'])
df.head()

,slots,frozen,ave,stdev,size
0,False,False,5.553422,0.322507,32
1,False,True,5.630893,0.201030,32
2,True,True,7.118935,0.183811,48
3,True,False,5.321728,0.235221,48


Looking at the table, we can say:

- `slots` are smaller and `frozen` is bigger
- `frozen` `slots` are  big and slow
- `slots` by them selves are the fastest
- Doing nothing, is really close to the fastest/smallest

Baically, `slots` only seem to bring limited performance improvement and `frozen` gives you decent performance at a larger memory foot print. However, **never ever** use `frozen` `slots` ... they suck! 